In [1]:
import json
from pathlib import Path
import sys

import pandas as pd
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error
import torch
import wandb

from chemprop import data, featurizers, models, nn

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
NUM_WORKERS = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles_std'
TARGET_COLUMNS = ['LogHLM', 'LogMLM', 'LogD', 'LogKSOL', 'LogMDR1-MDCKII']

In [4]:
def prepare_data(input_df):
    train_data, val_data = [], []
    for _, row in input_df.iterrows():
        dp = data.MoleculeDatapoint.from_smi(row[smiles_column], row[TARGET_COLUMNS].values)
        if row['split'] == 'train':
            train_data.append(dp)
        elif row['split'] == 'val':
            val_data.append(dp)

    pred_data = []
    for _, row in input_df.iterrows():
        dp = data.MoleculeDatapoint.from_smi(row[smiles_column], row[TARGET_COLUMNS].values)
        pred_data.append(dp)

    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

    train_dset = data.MoleculeDataset(train_data, featurizer)
    # scaler = train_dset.normalize_targets()

    val_dset = data.MoleculeDataset(val_data, featurizer)
    # val_dset.normalize_targets(scaler)

    pred_dset = data.MoleculeDataset(pred_data, featurizer)

    return train_dset, val_dset, pred_dset

In [5]:
# def train_model(config, train_dset, val_dset, num_workers, scaler):
def train_model(config, train_dset, val_dset, num_workers, save_dir):
    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])
    dropout = 0.2

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth, dropout=dropout)
    agg = nn.MeanAggregation()
    # output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    # ffn = nn.RegressionFFN(
    #     n_tasks=len(target_columns),
    #     output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers,
    #     dropout=dropout
    # )
    ffn = nn.RegressionFFN(
        n_tasks=len(TARGET_COLUMNS),
        output_transform=None, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers,
        dropout=dropout
    )
    batch_norm = True
    metric_list = [nn.metrics.MAE(), nn.metrics.R2Score()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    ckpt_callback = ModelCheckpoint(
        save_top_k=0,
        save_last=True
    )

    exp_name = f"chemprop_run_{RUN_IDX}"
    logger = WandbLogger(
        project="admet-challenge",
        name=exp_name,
        prefix=f"{save_dir.stem}",
        save_dir=f"../wandb/{exp_name}"
    )

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=200, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        enable_progress_bar=True,
        callbacks=[ckpt_callback],
        default_root_dir=save_dir,
        logger=logger
    )

    try:
        trainer.fit(model, train_loader, val_loader)
    except Exception as e:
        logger.finalize("failed")
        wandb.finish()
        raise e
    else:
        logger.finalize("success")

    return model

def predict(model, pred_dset, num_workers):
    pred_loader = data.build_dataloader(pred_dset, num_workers=num_workers, shuffle=False)
    
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        enable_progress_bar=True
    )

    model.eval()
    preds = trainer.predict(model, pred_loader, return_predictions=True)
    preds = torch.cat(preds)

    return preds

In [6]:
MODEL_CONFIG = {
    "depth": 5,
    "ffn_hidden_dim": 1500,
    "ffn_num_layers": 2,
    "message_hidden_dim": 300
}

In [7]:
# copypaste from https://github.com/asapdiscovery/asap-polaris-blind-challenge-examples/blob/a613051bac57060f686d9993e201ecaa15e51009/evaulation.py
# with a log-transform fix according to this issue https://github.com/asapdiscovery/asap-polaris-blind-challenge-examples/issues/14

from collections import defaultdict
from typing import Tuple

def mask_nan(y_true, y_pred):
    mask = ~np.isnan(y_true)
    y_true = np.array(y_true)[mask]
    y_pred = np.array(y_pred)[mask]
    return y_true, y_pred

def eval_admet(preds: dict[str, list], refs: dict[str, list]) -> Tuple[dict[str, float], np.ndarray]:
    """
    Eval ADMET targets with MAE for pre-log10 transformed targets (LogD) and MALE  (MAE on log10 transformed dataset) on non-transformed data

    This provides a "relative" error metric that will not be as sensitive to the large outliers with huge errors. This is sometimes known as MALE.

    Parameters
    ----------
    preds : dict[str, list]
        Dictionary of predicted ADMET values.
    refs : dict[str, list]
        Dictionary of reference ADMET values.

    Returns
    -------
    dict[str, float]
        Returns a dictonary of summary statistics
    """
    keys = {
        "MLM",
        "HLM",
        "KSOL",
        "LogD",
        "MDR1-MDCKII",
    }
    # will be treated as is
    logscale_endpts = {"LogD"}

    collect = defaultdict(dict)

    for k in keys:
        if k not in preds.keys() or k not in refs.keys():
            raise ValueError("required key not present")

        ref, pred = mask_nan(refs[k], preds[k])

        if k in logscale_endpts:
            # already log10scaled
            mae = mean_absolute_error(ref, pred)
            r2 = r2_score(ref, pred)
        else:
            # clip to a detection limit
            # epsilon = 1e-8
            # pred = np.clip(pred, a_min=epsilon, a_max=None)
            # ref = np.clip(ref, a_min=epsilon, a_max=None)

            # transform both log10scale
            pred_log10s = np.log10(pred + 1.)
            ref_log10s = np.log10(ref + 1.)

            # compute MALE and R2 in log space
            mae = mean_absolute_error(ref_log10s, pred_log10s)
            r2 = r2_score(ref_log10s, pred_log10s)

        collect[k]["mean_absolute_error"] = mae
        collect[k]["r2"] = r2

    # compute macro average MAE
    macro_mae = np.mean([collect[k]["mean_absolute_error"] for k in keys])
    collect["aggregated"]["macro_mean_absolute_error"] = macro_mae

    # compute macro average R2
    macro_r2 = np.mean([collect[k]["r2"] for k in keys])
    collect["aggregated"]["macro_r2"] = macro_r2

    return collect

In [8]:
def extract_preds(preds: pd.DataFrame):
    preds_dict = {}
    for t in TARGET_COLUMNS:
        if t in ["LogHLM", "LogMLM", "LogKSOL", "LogMDR1-MDCKII"]:
            # transform back to non-log scale
            preds_dict[t[3:]] = np.power(10, preds.iloc[:, preds.columns.get_loc(f"pred_{t}")].values) - 1.
        else:
            preds_dict[t] = preds.iloc[:, preds.columns.get_loc(f"pred_{t}")].values
    
    return preds_dict

def extract_refs(refs: pd.DataFrame):
    refs_dict = {}
    for t in TARGET_COLUMNS:
        if t in ["LogHLM", "LogMLM", "LogKSOL", "LogMDR1-MDCKII"]:
            refs_dict[t[3:]] = refs.iloc[:, refs.columns.get_loc(t[3:])].values
        else:
            refs_dict[t] = refs.iloc[:, refs.columns.get_loc(t)].values
    
    return refs_dict

In [9]:
def train_and_eval(input_paths, save_dirs):
    for input_path, save_dir in zip(input_paths, save_dirs):
        print(f"Training and predicting on {input_path}")
        input_df = pd.read_csv(input_path)
        train_dset, val_dset, pred_dset = prepare_data(input_df)
        model = train_model(MODEL_CONFIG, train_dset, val_dset, NUM_WORKERS, save_dir)
        preds = predict(model, pred_dset, NUM_WORKERS)

        output_df = input_df.copy()
        output_df[["pred_" + t for t in TARGET_COLUMNS]] = preds
        save_dir.mkdir(parents=True, exist_ok=True)
        output_df.to_csv(save_dir / "predictions.csv", index=False)

        train_preds = extract_preds(output_df[input_df["split"] == "train"])
        train_refs = extract_refs(input_df[input_df["split"] == "train"])
        val_preds = extract_preds(output_df[input_df["split"] == "val"])
        val_refs = extract_refs(input_df[input_df["split"] == "val"])

        metrics = eval_admet(train_preds, train_refs)
        print("Train metrics:")
        print(json.dumps(metrics, indent=2))

        metrics = eval_admet(val_preds, val_refs)
        print("\nVal metrics:")
        print(json.dumps(metrics, indent=2))
    
    wandb.finish()

## Run 0

In [10]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_0/split_{k}') for k in range(5)]
RUN_IDX = 0

In [11]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../data/asap/datasets/rnd_splits/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vladvin111 (vladvin-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "HLM": {
    "mean_absolute_error": 0.09867371808171951,
    "r2": 0.9432927322023731
  },
  "KSOL": {
    "mean_absolute_error": 0.12673047510108182,
    "r2": 0.9296293559254742
  },
  "LogD": {
    "mean_absolute_error": 0.14292655115558517,
    "r2": 0.9740211076476033
  },
  "MLM": {
    "mean_absolute_error": 0.09335113574107923,
    "r2": 0.962215751885825
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.08592744433543735,
    "r2": 0.9103767116892125
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.10952186488298062,
    "macro_r2": 0.9439071318700977
  }
}

Val metrics:
{
  "HLM": {
    "mean_absolute_error": 0.2367606982574419,
    "r2": 0.7500237855680927
  },
  "KSOL": {
    "mean_absolute_error": 0.34120478867360166,
    "r2": 0.4849545662377265
  },
  "LogD": {
    "mean_absolute_error": 0.42033029765793767,
    "r2": 0.6872220257853706
  },
  "MLM": {
    "mean_absolute_error": 0.22712209745536455,
    "r2": 0.619491074066433
  },
  "MDR1-

split_0-epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇████
split_0-train_loss_epoch,█▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▅▃▄▃▃▃▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▂▁
split_0-val/mae,█▅▂▂▃▂▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▅▆▇▇█▇█████████████████████████████████
split_0-val_loss,█▃▃▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇██
split_1-train_loss_epoch,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,██▃▅▄▄▂▂▃▃▂▂▂▂▃▂▁▃▁▁▁▁▁▂
split_1-val/mae,██▄▅▂▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▄▅▇▆▃▇▇█▇██████████████████████████████


## Cleaning up + run 1

In [12]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & (input_df["split"] == "train"), t] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [13]:
output_dir = Path("../output/asap/rnd_splits/chemprop/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [14]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [15]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            133
LogMLM            131
LogD               90
LogKSOL            69
LogMDR1-MDCKII      9
dtype: int64

In [16]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            183
LogMLM            178
LogD              127
LogKSOL           101
LogMDR1-MDCKII     43
dtype: int64

In [17]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2"

In [18]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_0/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "HLM": {
    "mean_absolute_error": 0.1929749891579298,
    "r2": 0.6956632670952683
  },
  "KSOL": {
    "mean_absolute_error": 0.204637572533364,
    "r2": 0.7705643108894307
  },
  "LogD": {
    "mean_absolute_error": 0.14971826881108444,
    "r2": 0.9762470290559407
  },
  "MLM": {
    "mean_absolute_error": 0.19545392495063757,
    "r2": 0.7351926621277907
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.10353594134833215,
    "r2": 0.8310309516372196
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.16926413936026957,
    "macro_r2": 0.80173964416113
  }
}

Val metrics:
{
  "HLM": {
    "mean_absolute_error": 0.22670592694709116,
    "r2": 0.730584551414741
  },
  "KSOL": {
    "mean_absolute_error": 0.33110204295999424,
    "r2": 0.5503752026893798
  },
  "LogD": {
    "mean_absolute_error": 0.3862548927059679,
    "r2": 0.7604336225141557
  },
  "MLM": {
    "mean_absolute_error": 0.19070134076387804,
    "r2": 0.7363095091191427
  },
  "MDR1-MDCK

split_0-epoch,▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
split_0-train_loss_epoch,█▇▆▅▄▄▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▅▃▃▃▄▂▂▂▁▂▃▁▁▄▂▂▁▁▁▂▁▁▁
split_0-val/mae,█▇▃▃▂▂▁▂▂▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▃▅█▇███████████████████████████████████
split_0-val_loss,█▆▅▅▄▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
split_1-train_loss_epoch,█▅▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▆▃▄▂▃▂▂▁▂▁▄▁▂▂▁▁▁▁▁▂▁▁▁
split_1-val/mae,█▅▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▇▇▄▇███████████████████████████████████


## Cleaning up + run 2

In [19]:
output_dir = Path("../output/asap/rnd_splits/chemprop/run_1/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [20]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [21]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_1/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_2/split_{k}') for k in range(5)]
RUN_IDX = "2_clean_worst_pct_0.2"

In [22]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_1/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "HLM": {
    "mean_absolute_error": 0.21051082981188016,
    "r2": 0.6477662833112732
  },
  "KSOL": {
    "mean_absolute_error": 0.20508622576620647,
    "r2": 0.7185849406955473
  },
  "LogD": {
    "mean_absolute_error": 0.15880126084628066,
    "r2": 0.9723041427490876
  },
  "MLM": {
    "mean_absolute_error": 0.2149739986894923,
    "r2": 0.6695698087155029
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.10423581351025271,
    "r2": 0.8101463018661609
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.17872162572482247,
    "macro_r2": 0.7636742954675144
  }
}

Val metrics:
{
  "HLM": {
    "mean_absolute_error": 0.22747329431416158,
    "r2": 0.7277862316327616
  },
  "KSOL": {
    "mean_absolute_error": 0.30488156221100593,
    "r2": 0.6064602479697583
  },
  "LogD": {
    "mean_absolute_error": 0.4320229924808848,
    "r2": 0.7235035054363963
  },
  "MLM": {
    "mean_absolute_error": 0.2299323856019651,
    "r2": 0.6453752257080578
  },
  "MDR1-

split_0-epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
split_0-train_loss_epoch,█▅▅▅▅▄▃▃▃▂▃▃▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▆▅▄▃▄▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁
split_0-val/mae,█▅▄▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▃▃▇▇▇█▇▇▇██████████████████████████████
split_0-val_loss,██▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
split_1-train_loss_epoch,█▄▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▅▄▃▃▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▂▁
split_1-val/mae,█▇▄▃▆▂▂▂▃▃▇▂▂▂▁▂▂▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▆▇▇▇█▅▇████████████████████████████████


## Cleaning up stero impure + run 1

In [23]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[
                input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & \
                    ~input_df["smiles_ext"].isna() & \
                    (input_df["split"] == "train"),
                t
            ] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [24]:
output_dir = Path("../output/asap/rnd_splits/chemprop/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [25]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [26]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            196
LogMLM            189
LogD              139
LogKSOL           106
LogMDR1-MDCKII     54
dtype: int64

In [27]:
tmp = pd.read_csv(input_paths[0])
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            99
LogMLM            82
LogD              72
LogKSOL           55
LogMDR1-MDCKII    31
dtype: int64

In [28]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            203
LogMLM            193
LogD              142
LogKSOL           108
LogMDR1-MDCKII     55
dtype: int64

In [29]:
tmp = pd.read_csv(output_dir / input_paths[0].name)
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            99
LogMLM            82
LogD              72
LogKSOL           55
LogMDR1-MDCKII    31
dtype: int64

In [30]:
input_paths = [Path(f'../output/asap/rnd_splits/chemprop/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2_stereo_impure"

In [31]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../output/asap/rnd_splits/chemprop/run_0/cleaned/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "HLM": {
    "mean_absolute_error": 0.23120698626968167,
    "r2": 0.621171273844568
  },
  "KSOL": {
    "mean_absolute_error": 0.2188618628347846,
    "r2": 0.7030738947092008
  },
  "LogD": {
    "mean_absolute_error": 0.17042093907639516,
    "r2": 0.9683062611028089
  },
  "MLM": {
    "mean_absolute_error": 0.23306752315626506,
    "r2": 0.6523948749932373
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.11235601598504436,
    "r2": 0.7784004298288068
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.19318266546443413,
    "macro_r2": 0.7446693468957244
  }
}

Val metrics:
{
  "HLM": {
    "mean_absolute_error": 0.22784435197343636,
    "r2": 0.7084146291254457
  },
  "KSOL": {
    "mean_absolute_error": 0.32585138890254395,
    "r2": 0.5534068783959648
  },
  "LogD": {
    "mean_absolute_error": 0.4709483754047841,
    "r2": 0.6782900854867137
  },
  "MLM": {
    "mean_absolute_error": 0.22497679639352564,
    "r2": 0.6590836881388589
  },
  "MDR1-

split_0-epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
split_0-train_loss_epoch,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,██▅▅▃▅▃▃▃▃▁▃▂▂▁▂▂▁▁▁▂▂▁▁
split_0-val/mae,█▄▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▁▄▅▆▇███████████████████████████████████
split_0-val_loss,█▇▆▅▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇█
split_1-train_loss_epoch,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▇▄▃▃▄▂▂▁▂▁▂▁▁▂▁▁▂▁▁▁▁▁▁
split_1-val/mae,█▆▆▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▃▅▇▇███████████████████████████████████


## Removing all stereo impure + run 0

In [32]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
output_dir = Path("../data/asap/datasets/rnd_splits/stereo_pure")
output_dir.mkdir(parents=True, exist_ok=True)

for input_path in input_paths:
    input_df = pd.read_csv(input_path)
    input_df = pd.concat([
        input_df[(input_df["split"] == "train") & input_df["smiles_ext"].isna()],
        input_df[input_df["split"] == "val"]
    ])
    input_df.to_csv(output_dir / input_path.name, index=False)

In [33]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/stereo_pure/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/chemprop/run_0/stereo_pure/split_{k}') for k in range(5)]
RUN_IDX = "0_stereo_pure"
train_and_eval(input_paths, save_dirs)

Training and predicting on ../data/asap/datasets/rnd_splits/stereo_pure/split_0.csv


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "HLM": {
    "mean_absolute_error": 0.12300280381519449,
    "r2": 0.9296002287064283
  },
  "KSOL": {
    "mean_absolute_error": 0.14930547514597278,
    "r2": 0.9259123882417241
  },
  "LogD": {
    "mean_absolute_error": 0.24441830903984776,
    "r2": 0.9480099263803069
  },
  "MLM": {
    "mean_absolute_error": 0.12194076109026604,
    "r2": 0.9415784568687966
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.11710970416301417,
    "r2": 0.8142946233279669
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.15115541065085905,
    "macro_r2": 0.9118791247050446
  }
}

Val metrics:
{
  "HLM": {
    "mean_absolute_error": 0.3490380909783692,
    "r2": 0.3816256264563448
  },
  "KSOL": {
    "mean_absolute_error": 0.45525179359768153,
    "r2": 0.2489808670527206
  },
  "LogD": {
    "mean_absolute_error": 0.8078668710792607,
    "r2": 0.23443671938275423
  },
  "MLM": {
    "mean_absolute_error": 0.4713732657418503,
    "r2": -0.34105081489976885
  },
  "MD

split_0-epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
split_0-train_loss_epoch,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▇▄▄▂▂▃▂▂▂▁▂
split_0-val/mae,█▇▇▆▅▂▁▁▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-val/r2,▂▁▃▄▇███████████████████████████████████
split_0-val_loss,█▆▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
split_1-train_loss_epoch,█▅▄▃▃▃▂▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▅▄▄▂▂▂▂▁▂▁▁
split_1-val/mae,█▇▆▆▆▃▃▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-val/r2,▁▁▁▂▂▃▄▆▆███████████████████████████████
